# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import time
import json

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities_output.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points("Lng", "Lat", geo= True, tiles = "OSM", frame_height=600, frame_width=850)

# Display the map
map_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_snow_city_df = city_data_df
ideal_snow_city_df = ideal_snow_city_df[(ideal_snow_city_df["Max Temp"] <10)]
ideal_snow_city_df = ideal_snow_city_df[(ideal_snow_city_df["Cloudiness"] >50)]
ideal_snow_city_df = ideal_snow_city_df[(ideal_snow_city_df["Wind Speed"] <10)]


# Display sample data
ideal_snow_city_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
8,8,sisimiut,66.9395,-53.6735,2.72,98,100,3.79,GL,1666108235
11,11,nikolskoye,59.7035,30.7861,8.94,90,100,3.22,RU,1666108237
13,13,dobryanka,58.4619,56.4165,5.63,93,100,3.55,RU,1666108237


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_snow_city_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["hotel_name"] = ""

# Display sample data
hotel_df.head(5)

,City,Country,Lat,Lng,Humidity,hotel_name
3,jamestown,US,42.0970,-79.2353,77,
5,ushuaia,AR,-54.8000,-68.3000,81,
8,sisimiut,GL,66.9395,-53.6735,98,
11,nikolskoye,RU,59.7035,30.7861,90,
13,dobryanka,RU,58.4619,56.4165,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
params = {
    "categories": category,
    "apikey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"]
    longitude = hotel_df["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
dukat - nearest hotel: No hotel found
senneterre - nearest hotel: No hotel found
midwest city - nearest hotel: No hotel found
chokurdakh - nearest hotel: No hotel found
klaksvik - nearest hotel: No hotel found
clyde river - nearest hotel: No hotel found
dikson - nearest hotel: No hotel found
ust-kulom - nearest hotel: No hotel found
udachnyy - nearest hotel: No hotel found
grindavik - nearest hotel: No hotel found
barrow - nearest hotel: No hotel found
saskylakh - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
kedrovyy - nearest hotel: No ho

,City,Country,Lat,Lng,Humidity,hotel_name,Hotel Name
3,jamestown,US,42.0970,-79.2353,77,,No hotel found
5,ushuaia,AR,-54.8000,-68.3000,81,,No hotel found
8,sisimiut,GL,66.9395,-53.6735,98,,No hotel found
11,nikolskoye,RU,59.7035,30.7861,90,,No hotel found
13,dobryanka,RU,58.4619,56.4165,93,,No hotel found
...,...,...,...,...,...,...,...
556,sorland,NO,67.6670,12.6934,67,,No hotel found
557,saryg-sep,RU,51.5000,95.6000,44,,No hotel found
570,staryy nadym,RU,65.6118,72.6842,90,,No hotel found
572,sovetskiy,RU,61.3614,63.5842,96,,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat", 
    geo=True, 
    tiles = "OSM", 
    frame_height=600, frame_width=850, 
    hover_cols=["City", "Hotel Name", "Country"]
    )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)